# 교통 표지판 이미지 분류 (pytorch)

차량 운전자의 기본 소양은 내 차의 주변 상황을 실시간으로 인지하면서 시시각각 변하는 도로 상황 속에서 사고가 나지 않도록 안전하게 운전하는 것일 것입니다. 이 주변 상황을 객관적으로 알려주는 장치에는 **교통 표지판**이 있을 것입니다. 교통 표지판은 앞선 도로의 상황이 어떨 것인지, 해당 도로에서는 어떤 점을 주의해야 하는지 등의 정보를 담고 있기에 초보 운전 때부터 모든 내용을 숙지하고 도로로 나오는 것이 바람직할 것입니다.

<center><img src="img/traffic_sign.jpg"></center>

최근에는 인공지능 기술이 발전함에 따라 사람이 아닌 자동차가 스스로 운전할 수 있게 하는 자율주행 기술이 함께 발전하고 있습니다. 자율주행 기술의 핵심은 역시 자동차에 장착된 카메라가 영상으로 도로 상황을 포착하는 것일 것입니다. 이 영상에는 당연히 교통 표지판도 포함되어 있을 것이므로, **교통 표지판의 내용을 분류**한 다음에 자동차가 적절한 행동을 취하는 과정이 필요할 것입니다.

따라서 이번 프로젝트에서는 교통 표지판이 촬영된 이미지를 통해 교통 표지판이 어떤 내용인지 분류하는 모델을 딥러닝 모델을 통해 간단하게 구현할 것입니다. 

이를 위해 이미지 데이터 처리에 특화된 **Convolutional Neural Network(CNN)** 를 활용할 것입니다.

## 프로젝트 목표
- 이미지 데이터를 전처리하는 과정과 딥러닝 모델에 적용하기 위한 형태로 변환하는 과정을 학습
- 교통 표지판 이미지 데이터를 활용하여 CNN 모델로 표지판 종류를 예측하는 분류 모델 학습

## 데이터 출처
-  https://www.kaggle.com/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign

## 프로젝트 목차

1. **데이터 확인하기**: 프로젝트에 사용할 데이터셋 디렉토리가 어떤 구조를 가지는지 확인

2. **데이터 분석**: 데이터셋에 포함된 이미지 데이터셋들이 어떤 특징을 가지는지 분석

3. **데이터셋 구성**: 학습과 평가에 사용할 데이터셋을 클래스로 정의

4. **모델 학습**: 데이터셋을 이용해 간단한 CNN 모델을 학습후 성능 평가

5. **모델 성능 개선(TODO)**: 모델의 성능을 향상시키기 위해 다양한 방법 적용

## 01. 데이터 확인하기

### 필요한 라이브러리 불러오기

In [ ]:
import os

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image, ImageDraw
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from tqdm import tqdm

### 데이터 구조 확인하기

본 프로젝트에서 사용할 데이터셋은 data라는 디렉토리에 아래와 구조로 저장되어 있습니다.

```
data
└─ Train
└─ Train.csv
└─ Test
└─ Test.csv
└─ Meta
└─ Meta.csv
```

`Meta` 및 `Meta.csv`의 내용은 본 프로젝트에서는 사용할 필요가 없기 때문에 따로 다루지 않겠습니다.

`Train.csv` 및 `Test.csv`에서는 각각 `Train` 폴더의 이미지와 `Test` 폴더의 이미지들의 메타데이터를 담고 있습니다. 

바로 확인해보겠습니다.

In [ ]:
root_dir = "data"

train_metadata = pd.read_csv(os.path.join(root_dir, "Train.csv"))
test_metadata = pd.read_csv(os.path.join(root_dir, "Test.csv"))

train_metadata.head()

In [ ]:
test_metadata.head()

두 파일 모두 같은 컬럼을 가지고 있는 것을 확인할 수 있습니다. 각 컬럼의 내용은 다음과 같습니다.

- `Width`: 이미지 파일의 너비
- `Height`: 이미지 파일의 높이
- `Roi.X1`: 이미지 파일 내에서 실제 표지판이 존재하는 곳을 상자로 감쌌을 때 좌상단 X좌표
- `Roi.Y1`: 이미지 파일 내에서 실제 표지판이 존재하는 곳을 상자로 감쌌을 때 좌상단 Y좌표
- `Roi.X2`: 이미지 파일 내에서 실제 표지판이 존재하는 곳을 상자로 감쌌을 때 우하단 X좌표
- `Roi.Y2`: 이미지 파일 내에서 실제 표지판이 존재하는 곳을 상자로 감쌌을 때 우하단 Y좌표
- `ClassId`: 해당 이미지의 클래스 ID (0 ~ 42까지 존재)
- `Path`: 실제 이미지가 저장된 경로

`Roi` 컬럼의 내용은 이미지로 직접 확인해보면 아래와 같습니다. (참고로 RoI는 Region of Interest라는 뜻입니다)


In [ ]:
sample_metadata = train_metadata.iloc[28589, :]
sample_img = Image.open(os.path.join(root_dir, sample_metadata["Path"]))

roi_box = ImageDraw.Draw(sample_img)
roi_box.rectangle(
    (
        sample_metadata["Roi.X1"],  # 좌상단 X 좌표
        sample_metadata["Roi.Y1"],  # 좌상단 Y 좌표
        sample_metadata["Roi.X2"],  # 우하단 X 좌표
        sample_metadata["Roi.Y2"],  # 우하단 Y 좌표
    ),
    outline=(255, 0, 0),  # Bounding Box 색을 빨간색으로
    width=2,  # Bounding Box 선의 두께
)

plt.imshow(sample_img)

설명한 대로 표지판을 감싸는 Bounding Box를 그리는 것을 확인할 수 있습니다.

이 정보는 본 프로젝트의 목적인 이미지 **분류**에는 크게 필요하지 않지만, **객체 인식(Object Detection)** 작업을 할 때는 필수적인 정보입니다.

하지만 본 프로젝트에서는 이미지가 어떤 표지판의 이미지인지 분류하는 것이 목적이므로, 이 정보는 사용하지 않습니다.

## 02. 데이터 분석

### 데이터 셋 특징 확인

이제 CSV로 저장된 메타데이터를 통해 전체 데이터 셋에 있는 이미지 파일들이 어떤 특징을 가지는지 확인해 보겠습니다.

먼저 앞서 메타데이터를 확인해 본 결과 각 이미지 파일의 너비와 높이가 모두 다른 것처럼 보였는데, 좀 더 확인해 보겠습니다.

In [ ]:
train_metadata.head(n=20)

역시 모두 같은 크기가 아님을 알 수 있습니다. 딥러닝 모델에 적용하기 위해서는 모두 같은 크기의 이미지를 가져야 하기 때문에 하나의 값으로 통일하는 것이 좋습니다.

그러기 위해 먼저 학습 데이터에서 이미지 너비(Width)와 높이(Height)의 분포를 살펴보겠습니다.

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12, 8))
sns.set_palette("tab10")
sns.histplot(
    train_metadata,
    x="Width",
    kde=True,
    stat="percent",
    label="Width",
    ax=ax[0],
    color="r",
)
ax[0].legend()
sns.histplot(
    train_metadata,
    x="Height",
    kde=True,
    stat="percent",
    label="Height",
    ax=ax[1],
    color="b",
)
ax[1].legend()

In [ ]:
print(
    f"[너비] 평균: {train_metadata['Width'].mean()}\n"
    f"[너비] 중간값(median): {train_metadata['Width'].median()}\n"
    f"[너비] 표준편차: {train_metadata['Width'].std(ddof=0)}"
)
print("=" * 50)
print(
    f"[높이] 평균: {train_metadata['Height'].mean()}\n"
    f"[높이] 중간값(median): {train_metadata['Height'].median()}\n"
    f"[높이] 표준편차: {train_metadata['Height'].std(ddof=0)}"
)

### 이미지 사이즈 정의

대부분의 이미지 파일의 너비와 높이는 50 이하인 것으로 확인됩니다. 실제로도 중간값은 50보다 작은 값에 위치하기도 합니다.

이미지 크기를 하나로 통일하기 위해서 어느 정도 절충안이 필요해 보이는데, 이때는 보통 유명한 모델이나 데이터 셋이 사용하는 값을 사용하는 것이 일반적입니다.

대표적인 이미지 사이즈로는 224x224, 32x32 등이 있습니다. 이번 프로젝트에서는 32x32로 이미지 크기를 통일하겠습니다.

In [ ]:
img_height, img_width = 32, 32

## 03. 데이터셋 구성

이제 pytorch를 활용하여 모델 학습을 위한 학습 데이터셋과 테스트 데이터셋을 구성하도록 하겠습니다.

먼저 학습 데이터셋은 `Train` 디렉토리에 존재하며, 각 이미지는 클래스 ID 별 디렉토리에 나눠져서 저장되어 있습니다.

테스트 데이터 셋은 하나의 디렉토리에 모든 이미지 파일이 저장되어 있습니다.

두 데이터 셋의 구조가 약간 다르지만 csv 파일을 통해 이미지 파일의 경로와 클래스 ID를 얻을 수 있기 때문에 이를 활용하여 데이터 셋을 구성할 수 있습니다.

### 데이터 전처리

이미지 데이터를 불러와서 pytorch 모델에 입력할 수 있는 형태로 변환하는 과정은 변환으로 구성할 수 있습니다.

이 실습에서는 다음과 같은 변환을 적용할 것입니다.

1. transforms.Resize: 이미지의 크기를 32x32 픽셀로 조정합니다. 이는 모델에 입력되는 이미지 크기를 통일하기 위한 단계입니다.
2. transforms.ToTensor: 이미지 데이터를 PyTorch 텐서로 변환합니다. (자동으로 0~1 사이의 값으로 정규화됩니다)
3. transforms.Normalize: 이미지의 픽셀값 범위를 [0, 1]에서 [-1, 1]로 정규화하여 모델의 학습을 더 안정적으로 만듭니다.  
   평균(0.5)과 표준편차(0.5)를 사용하여 각 채널의 픽셀값을 조정합니다.

In [ ]:
transform = transforms.Compose(
    [
        transforms.Resize((img_height, img_width)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

### 데이터셋 클래스 정의

먼저, PyTorch에서 데이터 셋을 구성하기 위해 `Dataset` 클래스를 상속받아 사용자 정의 데이터 셋을 만들어 보겠습니다.

데이터 셋의 용량이 크지 않으므로 모든 이미지를 메모리에 미리 올려놓고 사용하겠습니다.

이는 일반적으로 데이터 셋의 크기가 작을 때 사용하는 방법이며, 데이터 셋의 크기가 클 경우에는 필요한 데이터만 불러오는 방식을 사용하는 것이 일반적입니다.

In [ ]:
class TrafficSignDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []

        for i in range(len(self.annotations)):
            img_path = os.path.join(self.root_dir, self.annotations.iloc[i, 7])
            image = Image.open(img_path)
            label = torch.tensor(int(self.annotations.iloc[i, 6]))
            if self.transform:
                image = self.transform(image)
            self.images.append(image)
            self.labels.append(label)

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        # 이미지와 레이블을 메모리에서 직접 반환
        image = self.images[index]
        label = self.labels[index]
        return image, label

### 데이터 셋 생성

이제 정의한 데이터셋 클래스를 이용하여 학습 데이터 셋과 테스트 데이터 셋을 생성해 보겠습니다.

생성하는 순간 모든 이미지 데이터가 메모리에 로드되므로, 약간의 시간이 소요될 수 있습니다.

In [ ]:
# 학습 데이터셋 생성
train_dataset = TrafficSignDataset(
    csv_file="data/Train.csv",
    root_dir="data",
    transform=transform,
)

# 테스트 데이터셋 생성
test_dataset = TrafficSignDataset(
    csv_file="data/Test.csv", root_dir="data", transform=transform
)

### 데이터 로더 생성

이제 이 데이터 셋에서 데이터를 불러와주는 데이터 로더를 생성해 보겠습니다.

데이터 로더는 데이터셋에서 데이터를 미니 배치 단위로 추출하여 모델에 입력해주는 역할을 합니다.

In [ ]:
# 학습 데이터 로더 생성
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=100,
    shuffle=True,
    num_workers=2,
)

# 테스트 데이터 로더 생성
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=100,
    shuffle=False,
    num_workers=2,
)

주의할 점은 학습용 데이터 셋은 순서를 섞어서 불러오는 것이 허용되고 일반적이지만, 테스트 데이터 셋은 가능한 동일한 결과를 얻기 위해 순서대로 불러오는 것이 좋습니다.

## 04. 모델 학습

이제 이 데이터를 이용해 PyTorch에서 간단한 CNN 모델을 정의하고 학습을 진행해 보겠습니다.

### 간단한 CNN 모델 정의

pytorch에서 모델을 정의할 때는 `torch.nn.Module`을 상속받아 클래스를 정의하고, `forward` 메소드에 모델의 연산을 정의해야 합니다.

forward 연산의 경우 입력 데이터를 받아 모델의 출력을 계산하고 반환하는 역할을 합니다.

모델의 구조와 별개로 forward 연산을 구성할 수 있지만, 가독성을 위해 모델의 구조와 최대한 동일하게 구성하는 것이 좋습니다.

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        # 특징 추출기
        self.extractor = nn.Sequential(
            nn.Conv2d(3, 5, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )

        # 분류기
        self.classifier = nn.Sequential(nn.Linear(5 * 16 * 16, 43))

    def forward(self, x):
        out = self.extractor(x)
        out = out.view(out.shape[0], -1)
        out = self.classifier(out)

        return out

### 모델 학습

이제 이 모델의 인스턴스를 생성하고 학습을 진행해 보겠습니다.

In [ ]:
model = CNN()  # 모델 생성

학습에 필요한 하이퍼파라미터를 정의하고, 손실 함수와 최적화 알고리즘을 정의합니다.

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

지금까지 정의한 내용을 바탕으로 모델을 학습시킵니다.

In [ ]:
num_epochs = 3

# 모델 학습
for epoch in tqdm(range(num_epochs)):
    for X_batch, y_batch in train_loader:
        # 순전파
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        # 역전파
        optimizer.zero_grad() # gradient 초기화
        loss.backward()
        optimizer.step()

### 테스트 성능 평가

방금 학습한 모델의 성능을 테스트 데이터셋을 사용하여 평가합니다.

In [ ]:
# 검증 모드로 변경
model.eval()

# 테스트 데이터에 대한 정확도 계산
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"테스트 정확도: {100 * correct / total:.3f}%")

아직 성능이 만족스럽지는 않습니다.

이는 다양한 원인이 있을 수 있습니다.

- 데이터셋의 크기가 작아서 학습이 제대로 이루어지지 않았을 수 있습니다.
- 학습률이나 다른 하이퍼파라미터가 잘못 설정되었을 수 있습니다.
- 데이터셋이 불균형하게 구성되어 있을 수 있습니다.
- 모델의 구조가 너무 복잡하거나 단순해서 학습이 제대로 이루어지지 않았을 수 있습니다.

그 외에도 다양한 원인이 있을 수 있습니다.

최적의 모델을 얻기 위해서는 다양한 시도가 필요합니다.

## 05. 모델 성능 개선

모델의 구조를 개선하고 하이퍼파라미터를 조정하여 성능을 높여보겠습니다.

오래 걸리는 학습을 위해 GPU를 사용하여 학습을 진행합니다.

### GPU 확인

복잡한 모델을 학습할 때는 GPU를 사용하는 것이 좋습니다.

현재 사용 가능한 GPU가 있는지 확인하고, 사용 가능하다면 해당 GPU를 사용할 수 있도록 설정합니다.

In [ ]:
# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

### 모델 구조 개선

앞에서 정의한 `CNN` 모델의 구조를 변경합니다.

유명한 모델인 ResNet의 구조를 참고하여 모델의 구조를 정의합니다.

In [ ]:
def conv_block(in_channels, out_channels, pool=False):
    layers = [
        nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
    ]
    if pool:
        layers.append(nn.MaxPool2d(2))
    return layers


class SimpleResnet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Sequential(
            *conv_block(3, 64),
            *conv_block(64, 128, pool=True),
        )
        self.res1 = nn.Sequential(*conv_block(128, 128), *conv_block(128, 128))

        self.conv2 = nn.Sequential(
            *conv_block(128, 256, pool=True),
            *conv_block(256, 512, pool=True),
        )
        self.res2 = nn.Sequential(*conv_block(512, 512), *conv_block(512, 512))

        self.classifier = nn.Sequential(
            nn.MaxPool2d(4),
            nn.Flatten(),
            nn.Dropout(0.2),
            nn.Linear(512, 43),
        )

    def forward(self, x):
        out = self.conv1(x)
        out = self.res1(out) + out
        out = self.conv2(out)
        out = self.res2(out) + out
        out = self.classifier(out)

        return out

### 유틸리티 클래스 정의

앞에서 사용한 코드를 보면 학습이 불편하고 복잡합니다. 또한 중간중간 결과를 확인하기도 어려운 구조로 되어 있습니다.

이를 일반화하여 학습을 편하게 진행하고 중간 결과를 확인하기 위해 유틸리티 클래스를 정의합니다.

In [ ]:
from typing import Callable


# 학습에 도움을 주는 다양한 함수를 모아둔 유틸리티 클래스
class ClassifyModel:
    def __init__(self, model: nn.Module):
        self.model = model  # 사용할 모델

    def compile(self, loss_func: Callable, opt, device: torch.device):
        "손실함수, 옵티마이저, 디바이스 설정"
        self.loss_func = loss_func
        self.opt = opt
        self.device = device
        self.model.to(device)

    def loss_batch(self, x, y):
        "손실값 계산 및 예측값 반환"

        pred_y = self.model.forward(x)
        return self.loss_func(pred_y, y), pred_y

    def correct_batch(self, pred_y, y):
        "예측값과 정답을 비교하여 올바르게 예측한 개수를 반환"

        _, y_pred = torch.max(pred_y, 1)
        return (y_pred == y).float().sum()

    def save_checkpoint(self, epoch: int, dir_path: str):
        "체크포인트 저장"
        path = os.path.join(dir_path, f"model_{epoch+1}.pth")
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": self.model.state_dict(),
                "optimizer_state_dict": self.opt.state_dict(),
            },
            path,
        )
        return path

    def fit(
        self,
        epochs,
        train_dl,
        valid_dl,
        save_ckpt=False,
        ckpt_dir="checkpoints",
    ):
        "모델 학습"

        for epoch in range(epochs):
            # 학습
            train_loss_sum = 0.0
            train_correct = 0  # 정답을 맞춘 개수
            self.model.train()
            for x, y in tqdm(train_dl):
                x, y = x.to(self.device), y.to(self.device)
                loss, pred_y = self.loss_batch(x, y)
                loss.backward()
                self.opt.step()
                self.opt.zero_grad()

                train_loss_sum += loss.item() * len(y)
                train_correct += self.correct_batch(pred_y, y)

            train_loss = train_loss_sum / len(train_dl.dataset)
            train_acc = train_correct / len(train_dl.dataset)

            # 검증
            val_loss_sum = 0.0
            val_acc_sum = 0
            self.model.eval()
            with torch.no_grad():
                # 검증 데이터셋에 대한 손실값 계산
                for x, y in valid_dl:
                    x, y = x.to(self.device), y.to(self.device)

                    batch_loss, pred_y = self.loss_batch(x, y)

                    val_loss_sum += batch_loss.item() * len(y)
                    val_acc_sum += self.correct_batch(pred_y, y)

                val_loss = val_loss_sum / len(valid_dl.dataset)
                val_acc = val_acc_sum / len(valid_dl.dataset)

            print(
                f"Epoch: {epoch+1}/{epochs} Train Loss: {train_loss:.4f} Train Acc: {train_acc:.4f} Val Loss: {val_loss:.4f} Val Acc: {val_acc:.4f}"
            )

            if save_ckpt:
                # 체크포인트 저장
                os.makedirs(ckpt_dir, exist_ok=True)
                path = self.save_checkpoint(epoch, ckpt_dir)
                print(f"Checkpoint saved at {path}")

    def parameters(self):
        "모델의 파라미터 반환"
        return self.model.parameters()

### batch size 변경

batch size를 크게 하면 더 많은 데이터를 한 번에 학습할 수 있습니다.

이는 더 빠른 학습을 가능하게 하지만, 메모리를 더 많이 사용하게 됩니다.

따라서 GPU 메모리가 부족하다면 batch size를 줄이거나, 더 많은 메모리를 사용할 수 있는 환경을 사용해야 합니다.

이 실습에서는 충분히 큰 GPU 메모리를 사용할 것이므로 더 빠른 학습을 위해 데이터의 batch size를 늘립니다.

In [ ]:
# 학습 데이터 로더 생성
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=256,
    shuffle=True,
    num_workers=4,
)

# 테스트 데이터 로더 생성
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=256,
    shuffle=False,
    num_workers=4,
)

### 개선된 모델 학습

이제 개선한 모델을 학습시키겠습니다.

학습을 진행하면서 각 epoch마다 학습 데이터와 테스트 데이터에 대한 loss값과 정확도를 확인합니다.

또한, 각 epoch마다 체크포인트를 저장하여 학습 중간에 중단되어도 다시 이어서 학습을 진행할 수 있도록 합니다.

특정 epoch에서 성능이 가장 좋았으며 이후 과적합이 발생했을 때, 해당 체크포인트를 불러와서 사용할 수 있습니다.

In [ ]:
num_epochs = 10

# 정의한 모델을 ClassifyModel로 감싸서 학습
model = ClassifyModel(SimpleResnet())

# 모델 컴파일
model.compile(
    loss_func=nn.CrossEntropyLoss(),
    opt=optim.Adam(model.parameters()),
    device=device,
)

# 모델 학습
model.fit(num_epochs, train_loader, test_loader, save_ckpt=True)

### 가장 좋은 성능의 모델 불러오기

앞에서 진행한 학습의 출력 내역을 보면 특정 epoch에서 가장 좋은 성능을 보였을 것입니다.

모든 epcoh의 가중치가 저장되어 있으므로 그중 가장 좋은 성능을 보인 모델을 불러와 테스트 데이터셋으로 성능을 확인합니다.

(이 실습에선 5 epoch에서 가장 좋은 성능을 보였다고 가정합니다.)

In [ ]:
best_model = SimpleResnet()
best_model.load_state_dict(
    torch.load("checkpoints/model_5.pth")["model_state_dict"]
)
best_model.eval()

# GPU로 이동
best_model = best_model.to(device)

# 테스트 데이터에 대한 정확도 계산
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in tqdm(test_loader):
        outputs = best_model(images.to(device))
        _, predicted = torch.max(outputs.to("cpu").data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"테스트 정확도: {100 * correct / total:.3f}%")

<span style="color:rgb(120, 120, 120)">본 학습 자료를 포함한 사이트 내 모든 자료의 저작권은 엘리스에 있으며 외부로의 무단 복제, 배포 및 전송을 불허합니다.

Copyright @ elice all rights reserved</span>